# Run this code to generate the creation_data folder, which is used by my analysis code to correct the years from the SAP timestamps.




## RUN THIS CODE IF YOU'RE ON THE LAMBDA COMPUTER:

## It asks the user for the:
- treatment date (YYYY-MM-DD), the treatment time (HH:MM, in 24 hr time), 
- then the treatment type (e.g. Area X bilateral lesion, DOI 10 mg/kg). 
- Then, point it to the animal's data folder (e.g. USA5508's big folder of data). The code looks through the folders of .wav files from SAP to find the day they were created (as well as the different days the .wav files were created, if applicable) so we can add the year of the recordings to the .wav files, since SAP only saves the month, date, and time of the recording.

In [ ]:
from pathlib import Path
import time
import json
from datetime import datetime

# Set the path to the main folder here
# Example: folder_path = "/path/to/your/main/folder"
folder_path = "/Volumes/MelissaSSD/DOI_Pre_Post_Recordings/DOI_USA5506"

# Function to validate user input for treatment date
def get_treatment_date():
    while True:
        treatment_date_input = input("Please enter the treatment date (YYYY-MM-DD): ")
        try:
            # Parse the input date to check if it's valid
            treatment_date = datetime.strptime(treatment_date_input, '%Y-%m-%d')
            return treatment_date.strftime('%Y-%m-%d')  # Return the date in the proper format
        except ValueError:
            print("Invalid date format. Please try again.")

# Function to validate user input for treatment time
def get_treatment_time():
    while True:
        treatment_time_input = input("Please enter the treatment time (HH:MM, 24-hour format): ")
        try:
            # Parse the input time to check if it's valid
            treatment_time = datetime.strptime(treatment_time_input, '%H:%M')
            return treatment_time.strftime('%H:%M')  # Return the time in the proper format
        except ValueError:
            print("Invalid time format. Please try again.")

# Function to get treatment type
def get_treatment_type():
    treatment_type = input("Please enter the treatment type: ")
    return treatment_type

# Get the treatment date, time, and type from the user
treatment_date = get_treatment_date()
treatment_time = get_treatment_time()
treatment_type = get_treatment_type()

# Check if a folder path was provided and if it exists
main_folder = Path(folder_path)
if main_folder.is_dir():
    # Dictionary to store subdirectory info
    folder_data = {
        "treatment_date": treatment_date,  # Store the treatment date entered by the user
        "treatment_time": treatment_time,  # Store the treatment time entered by the user
        "treatment_type": treatment_type,  # Store the treatment type entered by the user
        "subdirectories": {}
    }

    # Loop through each subdirectory in the main folder
    for subdirectory in main_folder.iterdir():
        if subdirectory.is_dir():  # Ensure it is a subdirectory
            # Get the last modification time of the subdirectory
            subdirectory_creation_time = subdirectory.stat().st_mtime
            subdirectory_creation_date = time.strftime('%Y-%m-%d', time.localtime(subdirectory_creation_time))

            # Set to store unique file modification dates within the subdirectory
            unique_mod_dates = set()

            # Loop through files within this subdirectory
            for file in subdirectory.rglob('*'):
                if file.is_file():
                    # Get the modification time of the file
                    file_mod_time = file.stat().st_mtime
                    
                    # Convert the timestamp to a date (not including time)
                    file_mod_date = time.strftime('%Y-%m-%d', time.localtime(file_mod_time))
                    
                    # Add the file modification date to the set
                    unique_mod_dates.add(file_mod_date)
            
            # Add the subdirectory modification date and file modification dates to the dictionary
            folder_data["subdirectories"][subdirectory.name] = {
                "subdirectory_creation_date": subdirectory_creation_date,
                "unique_file_creation_dates": sorted(list(unique_mod_dates))  # Convert set to a sorted list
            }

    # Save the data to a .json file with the main folder's name
    json_file_name = f"{main_folder.name}_treatment_and_recording_dates.json"
    json_file_path = main_folder / json_file_name

    with open(json_file_path, 'w') as json_file:
        json.dump(folder_data, json_file, indent=4)

    print(f"Data has been saved to {json_file_path}")
else:
    print("The specified path is not a valid directory.")


## Running on a Mac Option 1: If you run this cell, the will ask the the treatment date,time, and type, and .wav folder path for ONE bird. It will use a file explorer pop-up window for you to select the folder with the bird's data.

In [ ]:
# from pathlib import Path
# import time
# import json
# from tkinter import Tk
# from tkinter import filedialog
# from datetime import datetime

# # Function to validate user input for treatment date
# def get_treatment_date():
#     while True:
#         treatment_date_input = input("Please enter the treatment date (YYYY-MM-DD): ")
#         try:
#             # Parse the input date to check if it's valid
#             treatment_date = datetime.strptime(treatment_date_input, '%Y-%m-%d')
#             return treatment_date.strftime('%Y-%m-%d')  # Return the date in the proper format
#         except ValueError:
#             print("Invalid date format. Please try again.")

# # Function to validate user input for treatment time
# def get_treatment_time():
#     while True:
#         treatment_time_input = input("Please enter the treatment time (HH:MM, 24-hour format): ")
#         try:
#             # Parse the input time to check if it's valid
#             treatment_time = datetime.strptime(treatment_time_input, '%H:%M')
#             return treatment_time.strftime('%H:%M')  # Return the time in the proper format
#         except ValueError:
#             print("Invalid time format. Please try again.")

# # Function to get treatment type
# def get_treatment_type():
#     treatment_type = input("Please enter the treatment type: ")
#     return treatment_type

# # Get the treatment date, time, and type from the user
# treatment_date = get_treatment_date()
# treatment_time = get_treatment_time()
# treatment_type = get_treatment_type()

# # Initialize the folder_path variable
# folder_path = []

# # If folder_path is empty, open a GUI to let the user select a folder
# if not folder_path:
#     # Hide the main Tkinter window
#     Tk().withdraw()
    
#     # Open a dialog for folder selection
#     folder_path = filedialog.askdirectory(title="Select Main Folder")

# # Check if a folder was selected
# if folder_path:
#     # Path object for the selected folder
#     main_folder = Path(folder_path)

#     # Dictionary to store subdirectory info
#     folder_data = {
#         "treatment_date": treatment_date,  # Store the treatment date entered by the user
#         "treatment_time": treatment_time,  # Store the treatment time entered by the user
#         "treatment_type": treatment_type,  # Store the treatment type entered by the user
#         "subdirectories": {}
#     }

#     # Loop through each subdirectory in the main folder
#     for subdirectory in main_folder.iterdir():
#         if subdirectory.is_dir():  # Ensure it is a subdirectory
#             # Get the creation (birth) date of the subdirectory
#             subdirectory_creation_time = subdirectory.stat().st_birthtime
#             subdirectory_creation_date = time.strftime('%Y-%m-%d', time.localtime(subdirectory_creation_time))

#             # Set to store unique file creation (birth) dates within the subdirectory
#             unique_birth_dates = set()

#             # Loop through files within this subdirectory
#             for file in subdirectory.rglob('*'):
#                 if file.is_file():
#                     # Get the creation (birth) time of the file
#                     file_creation_time = file.stat().st_birthtime
                    
#                     # Convert the timestamp to a date (not including time)
#                     file_creation_date = time.strftime('%Y-%m-%d', time.localtime(file_creation_time))
                    
#                     # Add the file creation date to the set
#                     unique_birth_dates.add(file_creation_date)
            
#             # Add the subdirectory creation date and file creation dates to the dictionary
#             folder_data["subdirectories"][subdirectory.name] = {
#                 "subdirectory_creation_date": subdirectory_creation_date,
#                 "unique_file_creation_dates": sorted(list(unique_birth_dates))  # Convert set to a sorted list
#             }

#     # Save the data to a .json file with the main folder's name
#     json_file_name = f"{main_folder.name}_creation_data.json"
#     json_file_path = main_folder / json_file_name

#     with open(json_file_path, 'w') as json_file:
#         json.dump(folder_data, json_file, indent=4)

#     print(f"Data has been saved to {json_file_path}")
# else:
#     print("No folder was selected.")


## Running on a Mac 2: If you run this cell, it will do the same as the cell above (ask for the treatment day, time, and type for ONE bird) but, instead of using the file exporer pop-up window, it will have you write out the path to the folder containing the bird's recordings in line 8 (folder_path).

In [ ]:
# from pathlib import Path
# import time
# import json
# from datetime import datetime

# # Set the path to the main folder here
# # Example: folder_path = '/path/to/your/main/folder'
# folder_path = '/Volumes/ROSE2-SSD/RHV_song_counts/RC1_USA5506_Comp2'

# # Function to validate user input for treatment date
# def get_treatment_date():
#     while True:
#         treatment_date_input = input("Please enter the treatment date (YYYY-MM-DD): ")
#         try:
#             # Parse the input date to check if it's valid
#             treatment_date = datetime.strptime(treatment_date_input, '%Y-%m-%d')
#             return treatment_date.strftime('%Y-%m-%d')  # Return the date in the proper format
#         except ValueError:
#             print("Invalid date format. Please try again.")

# # Function to validate user input for treatment time
# def get_treatment_time():
#     while True:
#         treatment_time_input = input("Please enter the treatment time (HH:MM, 24-hour format): ")
#         try:
#             # Parse the input time to check if it's valid
#             treatment_time = datetime.strptime(treatment_time_input, '%H:%M')
#             return treatment_time.strftime('%H:%M')  # Return the time in the proper format
#         except ValueError:
#             print("Invalid time format. Please try again.")

# # Function to get treatment type
# def get_treatment_type():
#     treatment_type = input("Please enter the treatment type: ")
#     return treatment_type

# # Get the treatment date, time, and type from the user
# treatment_date = get_treatment_date()
# treatment_time = get_treatment_time()
# treatment_type = get_treatment_type()

# # Check if a folder path was provided and if it exists
# main_folder = Path(folder_path)
# if main_folder.is_dir():
#     # Dictionary to store subdirectory info
#     folder_data = {
#         "treatment_date": treatment_date,  # Store the treatment date entered by the user
#         "treatment_time": treatment_time,  # Store the treatment time entered by the user
#         "treatment_type": treatment_type,  # Store the treatment type entered by the user
#         "subdirectories": {}
#     }

#     # Loop through each subdirectory in the main folder
#     for subdirectory in main_folder.iterdir():
#         if subdirectory.is_dir():  # Ensure it is a subdirectory
#             # Get the creation (birth) date of the subdirectory
#             subdirectory_creation_time = subdirectory.stat().st_birthtime
#             subdirectory_creation_date = time.strftime('%Y-%m-%d', time.localtime(subdirectory_creation_time))

#             # Set to store unique file creation (birth) dates within the subdirectory
#             unique_birth_dates = set()

#             # Loop through files within this subdirectory
#             for file in subdirectory.rglob('*'):
#                 if file.is_file():
#                     # Get the creation (birth) time of the file
#                     file_creation_time = file.stat().st_birthtime
                    
#                     # Convert the timestamp to a date (not including time)
#                     file_creation_date = time.strftime('%Y-%m-%d', time.localtime(file_creation_time))
                    
#                     # Add the file creation date to the set
#                     unique_birth_dates.add(file_creation_date)
            
#             # Add the subdirectory creation date and file creation dates to the dictionary
#             folder_data["subdirectories"][subdirectory.name] = {
#                 "subdirectory_creation_date": subdirectory_creation_date,
#                 "unique_file_creation_dates": sorted(list(unique_birth_dates))  # Convert set to a sorted list
#             }

#     # Save the data to a .json file with the main folder's name
#     json_file_name = f"{main_folder.name}_creation_data.json"
#     json_file_path = main_folder / json_file_name

#     with open(json_file_path, 'w') as json_file:
#         json.dump(folder_data, json_file, indent=4)

#     print(f"Data has been saved to {json_file_path}")
# else:
#     print("The specified path is not a valid directory.")


In [6]:
from pathlib import Path
import time
import json
from datetime import datetime

# === Set the path to the main folder ===
# Example: folder_path = '/path/to/your/main/folder'
folder_path = '/Volumes/ROSE2-SSD/RHV_song_counts/RC7_USA5499_Comp2'

# === Function to validate user input for treatment date ===
def get_treatment_date():
    while True:
        treatment_date_input = input("Please enter the treatment date (YYYY-MM-DD): ")
        try:
            treatment_date = datetime.strptime(treatment_date_input, '%Y-%m-%d')
            return treatment_date.strftime('%Y-%m-%d')
        except ValueError:
            print("Invalid date format. Please try again.")

# === Function to validate user input for treatment time ===
def get_treatment_time():
    while True:
        treatment_time_input = input("Please enter the treatment time (HH:MM, 24-hour format): ")
        try:
            treatment_time = datetime.strptime(treatment_time_input, '%H:%M')
            return treatment_time.strftime('%H:%M')
        except ValueError:
            print("Invalid time format. Please try again.")

# === Function to get treatment type ===
def get_treatment_type():
    treatment_type = input("Please enter the treatment type: ")
    return treatment_type

# === Main script logic ===
def collect_treatment_metadata(folder_path):
    # Get the treatment metadata from the user
    treatment_date = get_treatment_date()
    treatment_time = get_treatment_time()
    treatment_type = get_treatment_type()

    # Convert to Path object
    main_folder = Path(folder_path)

    if not main_folder.is_dir():
        print("The specified path is not a valid directory.")
        return

    # Dictionary to store all data
    folder_data = {
        "treatment_date": treatment_date,
        "treatment_time": treatment_time,
        "treatment_type": treatment_type,
        "subdirectories": {}
    }

    # Loop through subdirectories
    for subdirectory in main_folder.iterdir():
        if subdirectory.is_dir():
            # Get subdirectory creation time
            subdirectory_creation_time = subdirectory.stat().st_birthtime
            subdirectory_creation_date = time.strftime('%Y-%m-%d', time.localtime(subdirectory_creation_time))

            unique_birth_dates = set()

            for file in subdirectory.rglob('*'):
                if file.is_file():
                    file_creation_time = file.stat().st_birthtime
                    file_creation_date = time.strftime('%Y-%m-%d', time.localtime(file_creation_time))
                    unique_birth_dates.add(file_creation_date)

            folder_data["subdirectories"][subdirectory.name] = {
                "subdirectory_creation_date": subdirectory_creation_date,
                "unique_file_creation_dates": sorted(list(unique_birth_dates))
            }

    # Save to JSON
    json_file_name = f"{main_folder.name}_creation_data.json"
    json_file_path = main_folder / json_file_name

    with open(json_file_path, 'w') as json_file:
        json.dump(folder_data, json_file, indent=4)

    print(f"Data has been saved to {json_file_path}")

# === Only run when script is executed directly ===
if __name__ == "__main__":
    collect_treatment_metadata(folder_path)


Data has been saved to /Volumes/ROSE2-SSD/RHV_song_counts/RC7_USA5499_Comp2/RC7_USA5499_Comp2_creation_data.json


# Use the following if tranferring files to a new computer or hard drive changes the folders's dates - user will need to adjust the code to have the years of recordings AND if you recorded the folders in the same year.

In [10]:
import os
import json
from pathlib import Path
from datetime import datetime

# === Filename parser ===
def extract_date_time_from_filename(filename, year):
    name, ext = os.path.splitext(filename)
    parts = name.split("_")
    
    if len(parts) < 7:
        return None  # skip unexpected formats

    try:
        # Last 5 parts represent: month, day, hour, minute, second
        month = int(parts[-5])
        day = int(parts[-4])
        hour = int(parts[-3])
        minute = int(parts[-2])
        second = int(parts[-1])
        date = datetime(year, month, day)
        time_str = f"{hour:02d}:{minute:02d}:{second:02d}"
        return date.strftime("%Y-%m-%d"), time_str
    except Exception as e:
        print(f"Error parsing {filename}: {e}")
        return None

# === Build JSON structure ===
def build_json_structure(folder_path, treatment_date, treatment_type, year):
    folder_path = Path(folder_path)
    
    output = {
        "treatment_date": treatment_date,
        "treatment_type": treatment_type,
        "subdirectories": {}
    }

    for subfolder in sorted(folder_path.iterdir()):
        if subfolder.is_dir():
            subfolder_name = subfolder.name
            creation_dates = set()

            for wav_file in subfolder.glob("*.wav"):
                result = extract_date_time_from_filename(wav_file.name, year)
                if result:
                    date_str, _ = result
                    creation_dates.add(date_str)

            if creation_dates:
                earliest_date = sorted(creation_dates)[0]
                output["subdirectories"][subfolder_name] = {
                    "subdirectory_creation_date": earliest_date,
                    "unique_file_creation_dates": sorted(creation_dates)
                }

    return output

# === USER-DEFINED VALUES HERE ===
main_folder = "/Volumes/my_own_SSD/UO_stuff/RC3_USA5510_Comp2"
animal_id = 'USA5510'
treatment_date = "2025-03-20"  # Must be in YYYY-MM-DD format
treatment_type = "Bilateral Area X lesion"
year = 2025  # Year when recordings were made

# === Run and Save ===
result_json = build_json_structure(main_folder, treatment_date, treatment_type, year)
output_file = Path(main_folder) / f"{animal_id}_creation_data.json"

with open(output_file, 'w') as f:
    json.dump(result_json, f, indent=4)

print(f"\n✅ Saved metadata JSON to: {output_file}")



✅ Saved metadata JSON to: /Volumes/my_own_SSD/UO_stuff/RC3_USA5510_Comp2/USA5510_creation_data.json


# Use the following if tranferring files changed their dates, AND if you recorded over multiple years (e.g. Nov 2023 - Feb 2024)

In [ ]:
import os
import json
from pathlib import Path
from datetime import datetime

# === Function to parse filenames ===
def extract_date_time_from_filename(filename, year):
    name, ext = os.path.splitext(filename)
    parts = name.split("_")
    
    if len(parts) < 7:
        return None

    try:
        month = int(parts[-5])
        day = int(parts[-4])
        hour = int(parts[-3])
        minute = int(parts[-2])
        second = int(parts[-1])
        date = datetime(year, month, day)
        time_str = f"{hour:02d}:{minute:02d}:{second:02d}"
        return date.strftime("%Y-%m-%d"), time_str
    except Exception as e:
        print(f"Error parsing {filename}: {e}")
        return None

# === Build JSON metadata structure ===
def build_json_structure(folder_path, folder_to_year_map, treatment_date, treatment_type):
    folder_path = Path(folder_path)
    
    output = {
        "treatment_date": treatment_date,
        "treatment_type": treatment_type,
        "subdirectories": {}
    }

    for subfolder in sorted(folder_path.iterdir()):
        if subfolder.is_dir():
            subfolder_name = subfolder.name
            year = folder_to_year_map.get(subfolder_name)
            
            if year is None:
                print(f"⚠️ Skipping folder '{subfolder_name}': No year specified.")
                continue

            creation_dates = set()

            for wav_file in subfolder.glob("*.wav"):
                result = extract_date_time_from_filename(wav_file.name, year)
                if result:
                    date_str, _ = result
                    creation_dates.add(date_str)

            if creation_dates:
                earliest_date = sorted(creation_dates)[0]
                output["subdirectories"][subfolder_name] = {
                    "subdirectory_creation_date": earliest_date,
                    "unique_file_creation_dates": sorted(creation_dates)
                }

    return output

# === USER-DEFINED VALUES HERE ===
main_folder = '/path/to/your/folder'  # Replace with your actual path

folder_year_map = {
    2023: ['0', '1', '2', '3'],
    2024: ['4', '5', '6', '7']
}

treatment_date = "2024-03-20"  # Must be in YYYY-MM-DD format
treatment_type = "Bilateral Area X lesion"

# === Flatten the year-to-folders map into folder-to-year map ===
flattened_map = {}
for year, folders in folder_year_map.items():
    for folder in folders:
        flattened_map[folder] = year

# === Run and Save ===
result_json = build_json_structure(main_folder, flattened_map, treatment_date, treatment_type)
output_file = Path(main_folder) / "creation_data.json"

with open(output_file, 'w') as f:
    json.dump(result_json, f, indent=4)

print(f"\n✅ Saved metadata JSON to: {output_file}")
